In [ ]:
from copy import deepcopy
from typing import Optional

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.figure import Figure
from numpy import sign


In [ ]:
coord = tuple[int, int]
col = set[int]
mat = dict[int, col]


In [ ]:
def read(file: str) -> set[coord]:
    """Read in a set of (x,y) coordinates."""
    inputs = set()
    with open(file) as f:
        for line in f:
            coords = line.strip().split(" -> ")
            for i, coord in enumerate(coords[:-1]):
                a = tuple(map(int, coord.split(",")))
                b = tuple(map(int, coords[i + 1].split(",")))
                if a[0] != b[0]:
                    r = range(a[0], b[0] + sign(b[0] - a[0]), sign(b[0] - a[0]))
                    inputs.update([(x, a[1]) for x in r])
                elif a[1] != b[1]:
                    r = range(a[1], b[1] + sign(b[1] - a[1]), sign(b[1] - a[1]))
                    inputs.update([(a[0], x) for x in r])
    return inputs


def set2dict(s: set[coord]) -> mat:
    return {x: set(y for xp, y in s if xp == x) for x in set(x for x, _ in s)}


In [ ]:
example = set2dict(read("inputs/day-14-example.txt"))
input = set2dict(read("inputs/day-14.txt"))


## Part 1

Simulate the falling sand.

In [ ]:
def simulate(x: int, y: int, input: mat, floor: Optional[int] = None) -> coord:
    # Off the edge
    if x not in input:
        if floor is None:
            return (-1, -1)
        input[x] = set()

    # Floor
    if floor is not None and floor not in input[x]:
        input[x].update({floor})

    # Off the edge
    if not any(z > y for z in input[x]):
        return (-1, -1)

    # Straight
    yfall = min(z for z in input[x] if z > y) - 1

    # Diagonal L
    if x - 1 not in input:
        if floor is None:
            return (-1, -1)
        input[x - 1] = {floor}

    if yfall + 1 not in input[x - 1] | {floor}:
        return simulate(x - 1, yfall + 1, input, floor)

    # Diagonal R
    if x + 1 not in input:
        if floor is None:
            return (-1, -1)
        input[x + 1] = {floor}

    if yfall + 1 not in input[x + 1] | {floor}:
        return simulate(x + 1, yfall + 1, input, floor)
    return (x, yfall)


In [ ]:
def runsim(inp: mat, floor: bool = False, cap: int = 100_000) -> tuple[int, mat]:
    inp = deepcopy(inp)
    ifloor = max(y for s in inp.values() for y in s) + 2 if floor else None

    n = 0
    for _ in range(cap):
        x, y = simulate(500, 0, inp, ifloor)
        if (x, y) == (-1, -1):
            break
        n += 1
        if (x, y) == (500, 0):
            break
        inp[x].update({y})

    return n, inp


In [ ]:
n1, i1 = runsim(input)


In [ ]:
def plot(m: mat) -> Figure:
    xmin = min(m)
    xmax = max(m)
    ymax = max(y for v in m.values() for y in v)

    grid = np.zeros((ymax + 1, xmax - xmin + 1))
    for x in m:
        for y in m[x]:
            grid[y, x - xmin] = 1

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(grid, cmap="gray_r")
    ax.set_frame_on(False)
    ax.set_xticks([])
    ax.set_yticks([])
    return fig


In [ ]:
plot(i1)
pass


## Part 2

Add a floor.

In [ ]:
n2, i2 = runsim(input, floor=True)


In [ ]:
n2


In [ ]:
plot(i2)
pass
